In [34]:
# using PyCall
# sage_all = pyimport("sage.all")

In [35]:
#using MIPVerify
using Gurobi
using MAT
using IntervalArithmetic
using MIPVerify:prep_data_file
using MIPVerify:read_datasets
using MIPVerify:load_all_binary_images
using MIPVerify:get_image
using MIPVerify:get_label
using MIPVerify:Flatten
using Random
using IntervalArithmetic
using SetRounding
# sage_all = pyimport("sage.all")

# 创建 MixedIntegerLinearProgram
# p = sage_all.MixedIntegerLinearProgram(solver="ppl",maximization=true)
#p.solver_parameter("timelimit", 60)
#设置输入扰动为0.1


In [36]:
setprecision(BigFloat, 10)
FLOATMIN = floatmin(Float16)
RATIONALMIN = convert(Rational, BigFloat(floatmin(Float16)))
RATIONAL_ABS = Rational(BigInt(2)^-23)
RATIONAL_REL_UP = 1 + Rational(BigInt(2)^-10)
RATIONAL_REL_LO = 1 - Rational(BigInt(2)^-10)


16383//16384

In [37]:
typeof(RATIONAL_ABS)

Rational{BigInt}

In [38]:
INTERVAL_REL = Interval(RATIONAL_REL_LO ,RATIONAL_REL_UP )
INTERVAL_ABS = Interval(-RATIONAL_ABS , RATIONAL_ABS)

[-1//8388608, 1//8388608]

In [39]:
struct Interval_Value
    value_up::Rational
    value_lo::Rational
end

In [40]:
abstract type Layer end
abstract type NeuralNet end

In [41]:
function convert_to_rational(x::Array{<:AbstractFloat})
    precision = Dict(Float16 => 80, Float32 => 80)
    
    if eltype(x) in [Float16, Float32]
        setprecision(BigFloat, precision[eltype(x)])
    end
    
    return convert.(Rational, BigFloat.(x))
end


convert_to_rational (generic function with 1 method)

In [42]:
function convert_to_rational_itv(x::Array{<:AbstractFloat})
    precision = Dict(Float16 => 64, Float32 => 64)
    
    if eltype(x) in [Float16, Float32]
        setprecision(BigFloat, precision[eltype(x)])
    end
    
    return Interval.(convert.(Rational, BigFloat.(x)))
end

convert_to_rational_itv (generic function with 1 method)

In [43]:
struct Ra_Linear{T<:Rational,U<:Rational} <: Layer
    matrix::Array{T,2}
    bias::Array{U,1}

    function Ra_Linear{T,U}(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Rational,U<:Rational}
        (matrix_width, matrix_height) = size(matrix)
        bias_height = length(bias)
        @assert(
            matrix_height == bias_height,
            "Number of output channels in matrix, $matrix_height, does not match number of output channels in bias, $bias_height."
        )
        return new(matrix, bias)
    end

end

function Ra_Linear(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Rational,U<:Rational}
    Ra_Linear{T,U}(matrix, bias)
end

Ra_Linear

In [44]:
struct Linear{T<:Real,U<:Real} <: Layer
    matrix::Array{T,2}
    bias::Array{U,1}

    function Linear{T,U}(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Real,U<:Real}
        (matrix_width, matrix_height) = size(matrix)
        bias_height = length(bias)
        @assert(
            matrix_height == bias_height,
            "Number of output channels in matrix, $matrix_height, does not match number of output channels in bias, $bias_height."
        )
        return new(matrix, bias)
    end

end

function Linear(matrix::Array{T,2}, bias::Array{U,1}) where {T<:Real,U<:Real}
    Linear{T,U}(matrix, bias)
end

Linear

In [45]:
#抽象矩阵乘法操作
function matmul(x::Array{T,1},params::Ra_Linear{U,V}) where {T<:Interval,U<:Rational,V<:Rational}
    Weight=transpose(params.matrix)
    (matrix_height, matrix_width) = size(Weight)
    (input_height,) = size(x)
    @assert(
        matrix_width == input_height,
        "Number of values in input, $input_height, does not match number of values, $matrix_height that Linear operates on."
    )

    pre_value = Interval[]
    for i in 1:matrix_height
        tmp_itv = 0
        for j in 1:matrix_width
            #tmp_l = RATIONAL_LO*((RATIONAL_LO*Weight[i,j].lo*x_itv[j].lo-RATIONALMIN) + tmp_l) - RATIONALMIN 
            tmp_itv = INTERVAL_REL*(INTERVAL_REL*(Weight[i,j]*x[j])+INTERVAL_ABS + tmp_itv) +INTERVAL_ABS
        end
        tmp_itv = INTERVAL_REL*(tmp_itv+params.bias[i]) + INTERVAL_ABS
        push!(pre_value,tmp_itv)
    end
    return pre_value

end

matmul (generic function with 1 method)

In [46]:
#抽象矩阵乘法操作
function matmul_float(x::Array{T,1},params::Linear{U,V}) where {T<:Real,U<:Real,V<:Real}
    Weight=transpose(params.matrix)
    (matrix_height, matrix_width) = size(Weight)
    (input_height,) = size(x)
    @assert(
        matrix_width == input_height,
        "Number of values in input, $input_height, does not match number of values, $matrix_height that Linear operates on."
    )

    return transpose(params.matrix) * x .+ params.bias

end

matmul_float (generic function with 1 method)

In [47]:
function relu(x_itv::Interval)
    if(x_itv.hi<=0)
        return interval(0)
    elseif x_itv.lo >=0
        return x_itv
    else
        return interval(0,x_itv.hi)
    end
end

relu (generic function with 2 methods)

In [48]:
function relu(x::Real)
    if(x<=0)
        return 0
    else
        return x
    end
end

relu (generic function with 2 methods)

In [49]:
function get_matrix_params(
    param_dict::Dict{String, Any},
    layer_name::String,
    expected_size::NTuple{2, Int};
    matrix_name::String = "weight",
    bias_name::String = "bias",
    data_type::Type = Float32,
    trans2itv::Bool =false,
    debug_test::Bool = false,
    to_double::Bool = false
)::Layer

    param_weight = convert_to_rational(param_dict["$layer_name/$matrix_name"])
    param_bias = convert_to_rational(dropdims(param_dict["$layer_name/$bias_name"], dims=1))
    params = Ra_Linear(param_weight, param_bias)
    return params
end

get_matrix_params (generic function with 1 method)

In [50]:
function get_matrix_params_float(
    param_dict::Dict{String, Any},
    layer_name::String,
    expected_size::NTuple{2, Int};
    matrix_name::String = "weight",
    bias_name::String = "bias",
    data_type::Type = Float32,
    trans2itv::Bool =false,
    debug_test::Bool = false,
    to_double::Bool = false
)::Layer

    param_weight = Float16.(param_dict["$layer_name/$matrix_name"])
    param_bias = Float16.(dropdims(param_dict["$layer_name/$bias_name"], dims=1))
    params = Linear(param_weight, param_bias)
    return params
end

get_matrix_params_float (generic function with 1 method)

In [51]:
function get_example_network_params(name::String)
    if name == "F16MNIST24"
        nn = Ra_Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "mnist_dnn_fp16.mat") |> matread
        fc1 = get_matrix_params(param_dict, "fc1", (784, 24))
        push!(nn,fc1)
        fc2 = get_matrix_params(param_dict, "fc2", (24, 24))
        push!(nn,fc2)
        logits = get_matrix_params(param_dict, "logits", (24, 10))
        push!(nn,logits)
        return nn
    elseif name == "F16MNIST_INPUT77"
        nn = Ra_Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "resized_mnist77input_mnist_dnn_fp16.mat") |> matread
        fc1 = get_matrix_params(param_dict, "fc1", (49, 10))
        push!(nn,fc1)
        fc2 = get_matrix_params(param_dict, "fc2", (10, 10))
        push!(nn,fc2)
        logits = get_matrix_params(param_dict, "logits", (10, 10))
        push!(nn,logits)
        return nn
    end
end



get_example_network_params (generic function with 1 method)

In [52]:
function get_example_network_params_float(name::String)
    if name == "F16MNIST24"
        nn = Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "mnist_dnn_fp16.mat") |> matread
        fc1 = get_matrix_params_float(param_dict, "fc1", (784, 24))
        push!(nn,fc1)
        fc2 = get_matrix_params_float(param_dict, "fc2", (24, 24))
        push!(nn,fc2)
        logits = get_matrix_params_float(param_dict, "logits", (24, 10))
        push!(nn,logits)
        return nn
    elseif name == "F16MNIST_INPUT77"
        nn = Linear[]
        param_dict = prep_data_file(joinpath("weights", "mnist"), "resized_mnist77input_mnist_dnn_fp16.mat") |> matread
        fc1 = get_matrix_params_float(param_dict, "fc1", (49, 10))
        push!(nn,fc1)
        fc2 = get_matrix_params_float(param_dict, "fc2", (10, 10))
        push!(nn,fc2)
        logits = get_matrix_params_float(param_dict, "logits", (10, 10))
        push!(nn,logits)
        return nn
    end
end

get_example_network_params_float (generic function with 1 method)

In [53]:
function propagation(x::Vector{Interval{Rational{BigInt}}}, params::Vector{Ra_Linear}) 
    for (i, layer) in enumerate(params)
        x = matmul(x, layer)
        float_x = convert.(Interval{Float64},x)
        #println("\n")
        #println("the res for l$i: $float_x")
        if i < length(params)
            x = relu.(x)
        end
    end
    return x
end


propagation (generic function with 1 method)

In [54]:
function propagation_float(x::Vector{<:Real}, params::Vector{Linear}) 
    for (i, layer) in enumerate(params)
        x = matmul_float(x, layer)
        #println("the res for l$i: $x")
        if i < length(params)
            x = relu.(x)
        end
    end
    return x
end

propagation_float (generic function with 1 method)

In [55]:
nn77 = get_example_network_params("F16MNIST_INPUT77")
nn_float77 = get_example_network_params_float("F16MNIST_INPUT77")

3-element Vector{Linear}:
 Linear{Float16, Float16}(Float16[-0.02318 -0.02861 … 0.05246 0.0965; -0.1404 -0.02074 … -0.137 -0.12195; … ; 0.62 -0.175 … -0.1498 -0.2269; 0.0085 0.0647 … 0.0479 0.0753], Float16[0.0776, 0.322, 0.1536, 0.2285, 0.4648, -0.10443, -0.2776, -0.1274, 0.1511, 0.2183])
 Linear{Float16, Float16}(Float16[-0.9106 1.381 … 1.541 1.106; 0.6562 -0.7007 … -0.739 0.2156; … ; 0.8374 1.381 … -0.872 0.2961; 0.838 -1.06 … 1.127 0.623], Float16[0.6475, 0.3887, 0.01096, 0.2815, -0.04425, -0.3892, -0.4456, 0.2242, 0.807, -0.716])
 Linear{Float16, Float16}(Float16[0.3677 0.264 … -1.148 -0.8857; -1.298 0.801 … 0.8286 0.1746; … ; -1.069 0.3748 … -0.10614 1.106; 0.162 -1.589 … -0.1913 1.041], Float16[-0.636, 1.055, -0.891, 0.4197, 0.7583, 1.254, -0.676, 0.5684, -2.102, 0.3184])

In [56]:
# mnist = read_datasets("MNIST")
# index = 6
# sample_image = get_image(mnist.test.images, index)
# sample_label = get_label(mnist.test.labels, index)
# flat_img = Float32.(Flatten(4)(sample_image))
# ra_sample_image = convert_to_rational_itv(flat_img)
# output = propagation_float(flat_img, nn_float)
# output_itv = propagation(ra_sample_image, nn)

In [57]:
binary_file_path = "/home/aritifact/aritifact_for_cp24/FloatMIPVerify/resized_images/resized_mnist_images77_test.bin"
image_size = (7, 7)
num_images = 10000
images, labels = load_all_binary_images(binary_file_path, image_size, num_images)

index = 100
sample_image = images[index]
sample_image = reshape(images[index], (1, 7, 7, 1))
sample_label = labels[index]
flat_img = Float16.(Flatten(4)(sample_image))
ra_sample_image = convert_to_rational_itv(flat_img)
output = propagation_float(flat_img, nn_float77)
output_itv = propagation(ra_sample_image, nn77)

10-element Vector{Interval}:
  [-2.5452, -2.28163]₆₄
 [-12.7628, -12.4827]₆₄
  [-3.28614, -3.04766]₆₄
   [1.5977, 1.7861]₆₄
   [6.00049, 6.29665]₆₄
  [-0.46202, -0.197842]₆₄
  [-8.29833, -8.04036]₆₄
   [6.47106, 6.74108]₆₄
   [3.95177, 4.08235]₆₄
  [12.5128, 12.7219]₆₄

In [58]:
using IntervalArithmetic

struct SampleInfo
    index::Int
    output_itv::Vector{Interval}
    output::Vector{Float32}
end

function analyze_samples77()
    bounded_num = Int[]
    right_classified = Int[]
    wrong_classified = SampleInfo[]
    misclassified = 0
    for index in 1:10000
        try
            sample_image = images[index]
            sample_image = reshape(images[index], (1, 7, 7, 1))
            sample_label = labels[index]
            flat_img = Float16.(Flatten(4)(sample_image))
            ra_sample_image = convert_to_rational_itv(flat_img)
            output = propagation_float(flat_img, nn_float77)
            output_itv = propagation(ra_sample_image, nn77)

            # Check if the predicted label matches the true label
            if argmax(output) != sample_label + 1
                #println("Sample $index misclassified.")
                misclassified = misclassified+1
                continue
            end

            # Check if the output is within the computed interval
            @assert all(output .<= sup.(output_itv)) && all(output .>= inf.(output_itv))

            # Update statistics based on classification result
            push!(bounded_num, index)
            if argmax(output_itv) == argmax(output)
                push!(right_classified, index)
            else
                #println("Sample $index misclassified.")
                sample_wrong = SampleInfo(index, output_itv, output)
                #println(sample_wrong)
                push!(wrong_classified,sample_wrong)
            end
        catch e
            println("Error processing sample $index: $e")
        end
    end

    run_statistics = Dict(
        :bounded_num => bounded_num,
        :right_classified => right_classified,
        :wrong_classified => wrong_classified,
        :misclassified => misclassified
    )
    return run_statistics
end

run_statistics = analyze_samples77()

#统计结果4个

Dict{Symbol, Any} with 4 entries:
  :right_classified => [1, 2, 3, 4, 5, 6, 7, 8, 10, 11  …  9991, 9992, 9993, 99…
  :wrong_classified => SampleInfo[SampleInfo(496, Interval[[6.56748, 6.84123]₆₄…
  :bounded_num      => [1, 2, 3, 4, 5, 6, 7, 8, 10, 11  …  9991, 9992, 9993, 99…
  :misclassified    => 913

In [73]:
total = 10000-run_statistics[:misclassified]

9183

In [74]:
size(run_statistics[:bounded_num])

(9183,)

In [67]:
nn784 = get_example_network_params("F16MNIST24")
nn_float784 = get_example_network_params_float("F16MNIST24")
mnist = read_datasets("MNIST")

mnist:
  `train`: {LabelledImageDataset}
    `images`: 60000 images of size (28, 28, 1), with pixels in [0.0, 1.0].
    `labels`: 60000 corresponding labels, with 10 unique labels in [0, 9].
  `test`: {LabelledImageDataset}
    `images`: 10000 images of size (28, 28, 1), with pixels in [0.0, 1.0].
    `labels`: 10000 corresponding labels, with 10 unique labels in [0, 9].

In [68]:
function analyze_samples784()
    bounded_num = Int[]
    right_classified = Int[]
    wrong_classified = SampleInfo[]
    misclassified = 0
    
    for index in 1:10000
        try
            sample_image = get_image(mnist.test.images, index)
            sample_label = get_label(mnist.test.labels, index)
            flat_img = Float32.(Flatten(4)(sample_image))
            ra_sample_image = convert_to_rational_itv(flat_img)
            output = propagation_float(flat_img, nn_float784)
            output_itv = propagation(ra_sample_image, nn784)

            # Check if the predicted label matches the true label
            if argmax(output) != sample_label + 1
                #println("Sample $index misclassified.")
                misclassified = misclassified+1
                continue
            end

            # Check if the output is within the computed interval
            @assert all(output .<= sup.(output_itv)) && all(output .>= inf.(output_itv))

            # Update statistics based on classification result
            push!(bounded_num, index)
            if argmax(output_itv) == argmax(output)
                push!(right_classified, index)
            else
                #println("Sample $index misclassified.")
                sample_wrong = SampleInfo(index, output_itv, output)
                #println(sample_wrong)
                push!(wrong_classified,sample_wrong)
            end
        catch e
            println("Error processing sample $index: $e")
        end
    end

    run_statistics = Dict(
        :bounded_num => bounded_num,
        :right_classified => right_classified,
        :wrong_classified => wrong_classified,
        :misclassified => misclassified
    )
    return run_statistics
end

run_statistics = analyze_samples784()


Dict{Symbol, Any} with 4 entries:
  :right_classified => [1, 2, 3, 4, 5, 6, 8, 9, 10, 11  …  9991, 9992, 9993, 99…
  :wrong_classified => SampleInfo[SampleInfo(256, Interval[[-1.32921, -0.823443…
  :bounded_num      => [1, 2, 3, 4, 5, 6, 8, 9, 10, 11  …  9991, 9992, 9993, 99…
  :misclassified    => 817

In [72]:
total = 10000-run_statistics[:misclassified]

9183

In [69]:
size(run_statistics[:bounded_num])

(9183,)